In [13]:
import pandas as pd
import xml.etree.cElementTree as et
print('ready')

ready


In [4]:
parsedXML = et.parse( "test.xml" )
parsedXML

In [5]:
dfcols = ['name','email','phone','street']
df = pd.DataFrame(columns=dfcols)

In [6]:
def getvalueofnode( node ):
    return node.text if node is not None else None

In [7]:
for node in parsedXML.getroot():
    name = node.attrib.get('name')
    email = node.find('email')
    phone = node.find('phone')
    street = node.find('address/street')
    df = df.append( pd.Series( [name, getvalueofnode(email), getvalueofnode(phone), getvalueofnode(street)], index=dfcols) ,ignore_index=True)

df.head(5)

,name,email,phone,street
0,gokhan,gokhan@gmail.com,555-1234,None
1,mike,mike@gmail.com,None,None
2,john,john@gmail.com,555-4567,None
3,david,None,555-6472,Fifth Avenue


In [34]:
# http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

# Now try the big file
import requests

user_agent_url = 'http://www.user-agents.org/allagents.xml'
xml_data = requests.get(user_agent_url).content

class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = et.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

xml2df = XML2DataFrame(xml_data)
xml_dataframe = xml2df.process_data()
xml_dataframe.shape

(2459, 8)

In [36]:
# http://timhomelab.blogspot.com/2014/01/how-to-read-xml-file-into-dataframe.html

from lxml import objectify

path = 'test2.xml'
xml = objectify.parse(open(path))
root = xml.getroot()
root.getchildren()[0].getchildren()
df = pd.DataFrame(columns=('id', 'name'))

for i in range(0,4):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['id', 'name'], [obj[0].text, obj[1].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
df.head()

,id,name
0,1,First
1,2,Second
2,3,Third
3,4,Fourth


In [14]:
# https://eli.thegreenplace.net/2012/03/15/processing-xml-in-python-with-elementtree/
tree = et.ElementTree(file='test3.xml')
tree

In [15]:
# fetching root element
root = tree.getroot()
root.tag, root.attrib

('doc', {})

In [16]:
# Iterate over elements
for child_of_root in root:
    print(child_of_root.tag, child_of_root.attrib)

branch {'name': 'testing', 'hash': '1cdf045c'}
branch {'name': 'release01', 'hash': 'f200013e'}
branch {'name': 'invalid'}


In [17]:
# access individual elements
root[0].tag, root[0].text

('branch', '\n        text,source\n    ')

In [18]:
# examine elements
for elem in tree.iter():
    print(elem.tag, elem.attrib)

doc {}
branch {'name': 'testing', 'hash': '1cdf045c'}
branch {'name': 'release01', 'hash': 'f200013e'}
sub-branch {'name': 'subrelease01'}
branch {'name': 'invalid'}


In [19]:
for elem in tree.iter(tag='branch'):
    print( elem.tag, elem.attrib)

branch {'name': 'testing', 'hash': '1cdf045c'}
branch {'name': 'release01', 'hash': 'f200013e'}
branch {'name': 'invalid'}


In [21]:
for elem in tree.iterfind('branch/sub-branch'):
    print( elem.tag, elem.attrib)

sub-branch {'name': 'subrelease01'}
